In [1]:
import pandas as pd

### Volume Trend Line Chart (Dashboard 1)

In [5]:
import pandas as pd
from datetime import datetime, timedelta

def generate_volume_trend(view="24h"):
    # Load CSVs
    solver_df = pd.read_csv("data/solver_competitions.csv")
    tx_df = pd.read_csv("data/transaction.csv")

    # Merge on auctionId
    merged = pd.merge(solver_df, tx_df, on="auctionId", how="inner")

    # Compute volume
    merged["volume"] = merged[["buyAmountInUSD", "sellAmountInUSD"]].min(axis=1)
    merged["timestamp"] = pd.to_datetime(merged["timestamp"])

    # Filter by time range
    now = merged["timestamp"].max()  # Use latest timestamp in data
    if view == "24h":
        cutoff = now - timedelta(hours=24)
        freq = "1h"
    elif view == "7d":
        cutoff = now - timedelta(days=7)
        freq = "4h"
    else:
        cutoff = merged["timestamp"].min()
        freq = "1H"  # default bin size

    merged = merged[merged["timestamp"] >= cutoff]

    # Bin by time
    merged["time_bin"] = merged["timestamp"].dt.floor(freq)

    # Aggregate volume
    trend = merged.groupby("time_bin")["volume"].sum().reset_index()
    trend["time_bin"] = trend["time_bin"].dt.strftime("%Y-%m-%d %H:%M")
    trend.rename(columns={"time_bin": "timestamp"}, inplace=True)

    # Save to JSON
    output_file = f"out/volume_trend_{view}.json"
    trend.to_json(output_file, orient="records")
    print(f"Wrote {output_file}")

generate_volume_trend("24h")
generate_volume_trend("7d")

Wrote out/volume_trend_24h.json
Wrote out/volume_trend_7d.json


### Order Solver Time Difference (Dashboard 1)

In [13]:
import pandas as pd
from datetime import datetime
from pathlib import Path

def generate_order_solver_time_diff(
    solver_path="data/solver_competitions.csv",
    tx_path="data/transaction.csv",
    order_path="data/order.csv",
    output_path="out/order_solver_time_diff.json"
):
    # Load data
    solver_df = pd.read_csv(solver_path)
    tx_df = pd.read_csv(tx_path)
    order_df = pd.read_csv(order_path)

    # 🛠 Fix: clean timestamps before merge
    tx_df["timestamp"] = pd.to_datetime(tx_df["timestamp"], utc=True).dt.tz_localize(None)
    order_df["createdTimestamp"] = pd.to_datetime(order_df["createdTimestamp"], utc=True).dt.tz_localize(None)

    # Merge
    merged = pd.merge(solver_df, tx_df, on="auctionId", how="inner")
    merged = pd.merge(merged, order_df, left_on="orderId", right_on="id", how="inner")

    # Compute time difference in seconds
    merged["time_diff_sec"] = (merged["timestamp"] - merged["createdTimestamp"]).dt.total_seconds()
    merged = merged.dropna(subset=["time_diff_sec"])

    # Filter between 0–180s
    merged = merged[(merged["time_diff_sec"] >= 0) & (merged["time_diff_sec"] <= 180)]

    # Bin into ranges (10s buckets)
    bins = list(range(0, 190, 10))
    labels = [f"{i}-{i+10}" for i in bins[:-1]]
    merged["bucket"] = pd.cut(merged["time_diff_sec"], bins=bins, labels=labels, right=False)

    # Count per bucket
    result = merged.groupby("bucket", observed=True).size().reset_index(name="count")
    result = result.dropna()
    result["bucket"] = result["bucket"].astype(str)

    # Save output
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    result.to_json(output_path, orient="records", indent=2)
    print(f"[✓] Wrote {len(result)} bins to {output_path}")

# Run
generate_order_solver_time_diff()


[✓] Wrote 14 bins to out/order_solver_time_diff.json


### Solver Participation % (Dashboard 1)

In [21]:
import pandas as pd
from pathlib import Path

def generate_solver_participation(
    participation_path="data/solver_competitions.csv",
    solver_labels_path="data/solver.csv",
    output_path="out/solver_participation.json"
):
    df = pd.read_csv(participation_path)
    label_df = pd.read_csv(solver_labels_path)

    # Drop missing
    df = df.dropna(subset=["solverAddress", "auctionId"])

    # Total auctions
    total_auctions = df["auctionId"].nunique()

    # Unique auctions per solver
    participation = df.groupby("solverAddress")["auctionId"].nunique().reset_index(name="auctions")
    participation["participation_pct"] = (participation["auctions"] / total_auctions * 100).round(2)

    # Merge with labelName
    participation = pd.merge(participation, label_df, left_on="solverAddress", right_on="address", how="left")
    participation["name"] = participation["labelName"].fillna(participation["solverAddress"])

    # Sort and limit
    participation = participation.sort_values(by="participation_pct", ascending=False).head(20)

    # Rename for frontend
    participation = participation[["solverAddress", "name", "participation_pct"]]
    participation = participation.rename(columns={"solverAddress": "solver"})

    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    participation.to_json(output_path, orient="records", indent=2)
    print(f"[✓] Wrote {len(participation)} labeled solvers to {output_path}")

generate_solver_participation()

[✓] Wrote 20 labeled solvers to out/solver_participation.json


### Token Pair Treemap

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

def generate_token_pair_treemap_with_bins(
    comp_path="data/solver_competitions.csv",
    order_path="data/order.csv",
    token_labels_path="data/token_labels.csv",
    output_path="out/treemap_token_pair_volume.json"
):
    comp_df = pd.read_csv(comp_path)
    order_df = pd.read_csv(order_path)
    label_df = pd.read_csv(token_labels_path, on_bad_lines="skip")

    order_df = order_df.rename(columns={
        "buyAmountInUSD": "orderBuyUSD",
        "sellAmountInUSD": "orderSellUSD",
    })

    # Keep only relevant columns
    label_df = label_df[["address", "symbol"]]
    merged = pd.merge(comp_df, order_df, left_on="orderId", right_on="id", how="inner")
    merged = pd.merge(merged, label_df, left_on="sellTokenAddress", right_on="address", how="left").rename(columns={"symbol": "sellSymbol"})
    merged = pd.merge(merged, label_df, left_on="buyTokenAddress", right_on="address", how="left").rename(columns={"symbol": "buySymbol"})

    merged = merged.dropna(subset=["sellSymbol", "buySymbol", "buyAmountInUSD", "sellAmountInUSD"])
    merged["tokenPair"] = merged.apply(
        lambda row: "-".join(sorted([row["sellSymbol"], row["buySymbol"]])),
        axis=1
    )
    merged["volume"] = merged[["buyAmountInUSD", "sellAmountInUSD"]].min(axis=1)

    # Bin volumes
    bins = [0, 1000] + [1000 + i * 1000 for i in range(1, 11)] + [np.inf]
    labels = [f"{int(bins[i])}-{int(bins[i+1]) if bins[i+1] != np.inf else '10000+'}" for i in range(len(bins) - 1)]
    merged["volume_bin"] = pd.cut(merged["volume"], bins=bins, labels=labels, right=False)

    # Group by tokenPair
    grouped = merged.groupby("tokenPair")

    output = []
    for pair, group in grouped:
        bin_counts = group["volume_bin"].value_counts().reindex(labels, fill_value=0).to_dict()
        total_volume = group["volume"].sum()
        output.append({
            "tokenPair": pair,
            "value": total_volume,
            "binCounts": bin_counts
        })

    # Sort and keep top 100
    output = sorted(output, key=lambda x: x["value"], reverse=True)[:100]

    # Save
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    pd.DataFrame(output).to_json(output_path, orient="records", indent=2)
    print(f"[✓] Saved treemap data with bin counts to {output_path}")

generate_token_pair_treemap_with_bins()

[✓] Saved treemap data with bin counts to out/treemap_token_pair_volume.json
